# Q1: Authenticated Encryption

This solution implements the authenticated encryption scheme as specified in the assignment. It consists of three Python scripts:

1.  **`gen.py`**: Generates the pre-shared keys.

2.  **`bob.py`**: Acts as the TCP server, listening for a connection from Alice.

3.  **`alice.py`**: Acts as the TCP client, connecting to Bob.

The implementation adheres to all specified security requirements:

- **Key Generation**: `gen.py` creates a file named `pw` containing two symmetric keys:
    - **16-byte (128-bit) key** for AES-128 encryption.
    - **32-byte (256-bit) key** for HMAC-SHA256 authentication.

- **Confidentiality**: All messages are encrypted using **AES-128-CTR**. A new random 8-byte `nonce` is generated for each message.

- **Authenticity**: Message authenticity is ensured using **HMAC-SHA256**.

- **Scheme**: The system uses the **Encrypt-then-MAC** (EtM) scheme. The HMAC is calculated over the `nonce` and the `ciphertext` (`nonce + ciphertext`).

- **Replay Protection**: A **sequence number** (implemented as an 8-byte counter) is prepended to the plaintext before encryption to protect against replay attacks. Both Alice and Bob maintain separate `send` and `receive` counters to ensure messages are processed in the correct order (Encrypt-then-MAC).

To handle variable-length messages over TCP, the following custom protocol is used:

1.  **Send**:
    1.  Plaintext is prepended with the 8-byte sequence number: `plaintext = seq_num_bytes + message_bytes`.

    2.  This plaintext is encrypted with AES-CTR, producing a `ciphertext` and a `nonce`.
    
    3.  An HMAC is calculated over the `nonce + ciphertext`.
    
    4.  The final payload is constructed: `payload = nonce + ciphertext + mac`.
    
    5.  A 4-byte length prefix (the length of the `payload`) is sent, followed by the `payload` itself.
2.  **Receive**:
    
    1.  The receiver reads the 4-byte length prefix.
    
    2.  It then reads the exact number of bytes specified by the length prefix.
    
    3.  The received `payload` is split into `nonce`, `ciphertext`, and `mac`.
    
    4.  The HMAC is verified. If it fails, the program exits.
    
    5.  The `ciphertext` is decrypted using the `nonce`.
    
    6.  The 8-byte sequence number is extracted from the decrypted plaintext and verified against the expected sequence number. If it doesn't match, the program exits.
    
    7.  The original message is returned.

## How to Run the Q1 Solution

### Prerequisites

1.  Install the required library:
    ```bash
    pip install pycryptodome
    ```
2.  Save `gen.py`, `bob.py`, and `alice.py` in the same directory.


### Instructions

1.  **Open Terminal 1.**
2.  Generate the keys:
    ```bash
    python gen.py
    ```
3.  Start the server (it will start listening on the defined port):
    ```bash
    python bob.py
    ```
    *Output:* `Bob is listening on localhost:65432...`

4.  **Open Terminal 2.**
5.  Run the client:
    ```bash
    python alice.py
    ```

### Result

The conversation will execute in both terminals, and each will display `Conversation successful and complete.`

# Q2. Signing with RSA
### Question - P1
An attacker can forge a valid signature without knowing the private key.
They simply choose an arbitrary value $\sigma^{*}$ and compute the corresponding message:
$$
M^{*} = (\sigma^{*})^{e} \mod N
$$
The pair $(M^{*}, \sigma^{*})$ will be accepted as a valid signature.
This allows a valid message-signature pair to be created from scratch, without ever acessing the private keys.
In resume:
- The plain RSA signature scheme is vulnerable to existential forgery because the attacker can generate forged signatures for new messages without knowledge of the private key.

Text taken from the book "Serious Cryptography", page 188, from Breaking Textbook RSA Signatures:  
"One such attack involves a trivial forgery: upon noticing that $0^{d} \mod{n} = 0$, $1^{d} \mod{n} = 1$, and $(n-1)^{d} \mod{n} = n-1$, regardless of the value of the private key $d$, an attacker can forge signatures of 0, 1, or (n-1) without knowing $d$."

### Question - P2
The properties of the hash functions are we using to ensure that the previous attack
no longer works are:
- Pre-image resistance: Given a specific message $M$, an attacker shouldn't be able to find a different message $M' \ne M$ with the same hash:
$$
H(M') = H(M)
$$
- Colision-Resistance: It should be infeasible to find any two different messages $M1 \ne M2$ such that:
$$
H(M1) = H(M2)
$$  


FDH (Full Domain Hash) solves the forgery problem by hashing the message before signing it. Instead of signing the message directly, the signer computes the signature as:

$$
\sigma = H(M)^d \bmod N
$$

During verification, the verifier checks whether:

$$
\sigma^e \bmod N = H(M)
$$

Because cryptographic hash functions are preimage-resistant, an attacker cannot start from a chosen $\sigma'$ and find a message $M'$ such that:

$$
H(M') = (\sigma')^e \bmod N
$$


Thus, the hash function prevents the attacker from reversing the RSA operation to produce valid forgeries. Only someone with the private key $d$ can generate correct signatures, fixing the security weakness of naive RSA signatures.
